## Exercise 09
Train a deep MLP on the MNIST dataset and see if you can get over 98% precision. Just like in the last exercise of Chapter 9, try adding all the bells and whistles (i.e., save checkpoints, restore the last checkpoint in case of an interruption, add summaries, plot learning curves using TensorBoard, and so on).

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Import MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Build TF Graph
n_inputs = 28 * 28
n_hidden_1 = 256
n_hidden_2 = 128
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

In [4]:
from tensorflow.contrib.layers import fully_connected
with tf.name_scope('dnn'):
    hidden_1 = fully_connected(X, n_hidden_1, scope='hidden1')
    hidden_2 = fully_connected(hidden_1, n_hidden_2, scope='hidden2')
    outputs = fully_connected(hidden_2, n_outputs, scope='outputs', activation_fn=None)

In [5]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=outputs)
    loss = tf.reduce_mean(xentropy, name='loss')

In [6]:
learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [7]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(outputs, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [8]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
# Add Summary for tensorboard
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = "tf_logs/c10_exercise-{}".format(now)
acc_summary = tf.summary.scalar("Accuracy", accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [10]:
n_samples_per_class = 500
X_train_small = []
y_train_small = []
for value in np.unique(mnist.train.labels):
    indices = np.where(mnist.train.labels == value)
    np.random.shuffle(indices[0])
    for image in mnist.train.images[indices[0][:n_samples_per_class]]:
        X_train_small.append(image)
        y_train_small.append(value)
X_train_small = np.array(X_train_small)
y_train_small = np.array(y_train_small)

In [12]:
model_path = "/Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C0_exercise/dnn_model.ckpt"
n_epoch = 50
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        n_batchs = mnist.train.num_examples // batch_size;
        for iteration in range(n_batchs):
            X_batch, y_batch = mnist.train.next_batch(batch_size);
            # wirte summary
            summary_str = acc_summary.eval(feed_dict={X: X_batch, y: y_batch})
            step = epoch * n_batchs + iteration
            file_writer.add_summary(summary_str, step)
#             print(step)
            # train
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y:mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, " Test accuracy:", acc_test)
    save_path = saver.save(sess, model_path)
file_writer.flush()
file_writer.close()

0 Train accuracy: 0.128  Test accuracy: 0.1322
50 Train accuracy: 0.549  Test accuracy: 0.5534
100 Train accuracy: 0.7328  Test accuracy: 0.7412
150 Train accuracy: 0.7882  Test accuracy: 0.7993
200 Train accuracy: 0.8178  Test accuracy: 0.8271
250 Train accuracy: 0.834  Test accuracy: 0.8408
300 Train accuracy: 0.8464  Test accuracy: 0.8515
350 Train accuracy: 0.8528  Test accuracy: 0.8583
400 Train accuracy: 0.8592  Test accuracy: 0.8665
450 Train accuracy: 0.8652  Test accuracy: 0.872


In [13]:
test_image = mnist.test.images[0]
test_label = mnist.test.labels[0]
with tf.Session() as sess:
    saver.restore(sess, model_path)
    pred_output = outputs.eval(feed_dict={X:[test_image]})
    y_pred = np.argmax(pred_output, axis=1)
    print("Test predict:", y_pred, "Correct:", test_label)

INFO:tensorflow:Restoring parameters from /Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C0_exercise/dnn_model.ckpt
Test predict: [7] Correct: 7
